In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler,OrdinalEncoder
from sklearn.compose import ColumnTransformer,make_column_transformer
from sklearn.svm import SVR
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVR

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.decomposition import PCA
# Tree-based models
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,ExtraTreesRegressor,GradientBoostingRegressor,AdaBoostRegressor
# Neural Network
from sklearn.neural_network import MLPRegressor
# XGBoost
from xgboost import XGBRegressor

from sklearn.metrics import r2_score

In [2]:
df = pd.read_csv('final.csv')
df.drop(columns={'Other'},inplace=True)

In [3]:
df.sample(2)

,area,property_type,bedRoom,bathroom,balcony,sector,Price,Study_room,Servant_room,Pooja_room,floor,age,feature_score,furnish_score
1777,10000.0,flat,4,4,3+,81,4.20,0,1,0,High Floor,Relatively New,High,High
502,7230.0,flat,4,4,3+,85,2.03,0,0,0,Mid Floor,Relatively New,Low,Unfurnished


In [4]:
df.floor.isna().sum()

np.int64(0)

In [5]:
X = df.drop(columns=['Price'])
y = df['Price']

In [6]:
# Applying the log1p transformation to the target variable
y_transformed = np.log1p(y)

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=42)

### Encoding Technique First is to apply OneHotEncoding

In [8]:
columns_to_encode = ['feature_score','furnish_score','floor','sector','balcony','property_type','age']

In [9]:
ohe = OneHotEncoder()
ohe.fit(X[['feature_score','furnish_score','floor','sector','balcony','property_type','age']])

,"categories categories: 'auto' or a list of array-like, default='auto'Categories (unique values) per feature:- 'auto' : Determine categories automatically from the training data.- list : ``categories[i]`` holds the categories expected in the ith column. The passed categories should not mix strings and numeric values within a single feature, and should be sorted in case of numeric values.The used categories can be found in the ``categories_`` attribute... versionadded:: 0.20",'auto'
,"drop drop: {'first', 'if_binary'} or an array-like of shape (n_features,), default=NoneSpecifies a methodology to use to drop one of the categories perfeature. This is useful in situations where perfectly collinearfeatures cause problems, such as when feeding the resulting datainto an unregularized linear regression model.However, dropping one category breaks the symmetry of the originalrepresentation and can therefore induce a bias in downstream models,for instance for penalized linear classification or regression models.- None : retain all features (the default).- 'first' : drop the first category in each feature. If only one category is present, the feature will be dropped entirely.- 'if_binary' : drop the first category in each feature with two categories. Features with 1 or more than 2 categories are left intact.- array : ``drop[i]`` is the category in feature ``X[:, i]`` that should be dropped.When `max_categories` or `min_frequency` is configured to groupinfrequent categories, the dropping behavior is handled after thegrouping... versionadded:: 0.21 The parameter `drop` was added in 0.21... versionchanged:: 0.23 The option `drop='if_binary'` was added in 0.23... versionchanged:: 1.1 Support for dropping infrequent categories.",None
,"sparse_output sparse_output: bool, default=TrueWhen ``True``, it returns a :class:`scipy.sparse.csr_matrix`,i.e. a sparse matrix in ""Compressed Sparse Row"" (CSR) format... versionadded:: 1.2 `sparse` was renamed to `sparse_output`",True
,"dtype dtype: number type, default=np.float64Desired dtype of output.",<class 'numpy.float64'>
,"handle_unknown handle_unknown: {'error', 'ignore', 'infrequent_if_exist', 'warn'}, default='error'Specifies the way unknown categories are handled during :meth:`transform`.- 'error' : Raise an error if an unknown category is present during transform.- 'ignore' : When an unknown category is encountered during transform, the resulting one-hot encoded columns for this feature will be all zeros. In the inverse transform, an unknown category will be denoted as None.- 'infrequent_if_exist' : When an unknown category is encountered during transform, the resulting one-hot encoded columns for this feature will map to the infrequent category if it exists. The infrequent category will be mapped to the last position in the encoding. During inverse transform, an unknown category will be mapped to the category denoted `'infrequent'` if it exists. If the `'infrequent'` category does not exist, then :meth:`transform` and :meth:`inverse_transform` will handle an unknown category as with `handle_unknown='ignore'`. Infrequent categories exist based on `min_frequency` and `max_categories`. Read more in the :ref:`User Guide `.- 'warn' : When an unknown category is encountered during transform a warning is issued, and the encoding then proceeds as described for `handle_unknown=""infrequent_if_exist""`... versionchanged:: 1.1 `'infrequent_if_exist'` was added to automatically handle unknown categories and infrequent categories... versionadded:: 1.6 The option `""warn""` was added in 1.6.",'error'
,"min_frequency min_frequency: int or float, default=NoneSpecifies the minimum frequency below which a category will beconsidered infrequent.- If `int`, categories with a smaller cardinality will be considered infrequent.- If `float`, categories with a smaller cardinality than `min_frequency * n_samples` will be considered infrequent... versionadded:: 1.1 Read more in the :ref:`User Guide `.",None
,"max_catego

In [10]:
column_trans=make_column_transformer(
    (OneHotEncoder(categories=ohe.categories_,drop='first'),columns_to_encode),
    (StandardScaler(), ['bedRoom', 'bathroom', 'area']),
    # (OrdinalEncoder(),['property_type','age']),
    remainder='passthrough')

In [11]:
model_dict = {
    'linear_reg':LinearRegression(),
    'svr':SVR(),
    'ridge':Ridge(),
    'LASSO':Lasso(),
    'decision tree': DecisionTreeRegressor(),
    'random forest':RandomForestRegressor(),
    'extra trees': ExtraTreesRegressor(),
    'gradient boosting': GradientBoostingRegressor(),
    'adaboost': AdaBoostRegressor(),
    'mlp': MLPRegressor(),
    'xgboost':XGBRegressor()
}

In [12]:
# model_output = []
for model_name,model in model_dict.items():
    # model_output.append(scorer(model_name, model))
    pipeline = make_pipeline(column_trans,model)
    pipeline.fit(X_train,y_train)
    y_pred = pipeline.predict(X_test)
    print(f" model is {model_name} and r2_score is {r2_score(y_test,y_pred)} and error is {mean_absolute_error(np.expm1(y_test),np.expm1(y_pred))}")

 model is linear_reg and r2_score is 0.7697457752052876 and error is 0.773677706406429
 model is svr and r2_score is 0.8274896690418562 and error is 0.6306906119670084
 model is ridge and r2_score is 0.768656755876552 and error is 0.7705179855230144
 model is LASSO and r2_score is -0.00021697241587892435 and error is 1.5827672722146149
 model is decision tree and r2_score is 0.751643386395044 and error is 0.6878656209692454
 model is random forest and r2_score is 0.8388628812729146 and error is 0.5553655015806788
 model is extra trees and r2_score is 0.8360543487776315 and error is 0.5533808247946582
 model is gradient boosting and r2_score is 0.8153220297847901 and error is 0.6727384152328557
 model is adaboost and r2_score is 0.45098979991869137 and error is 1.2145279435722705
 model is mlp and r2_score is 0.8135356292624332 and error is 0.643970394579385
 model is xgboost and r2_score is 0.8469747412682693 and error is 0.5647105077342791


### Technique 2 Target Encoding and all 

In [13]:
import category_encoders as ce
# !pip install category_encoders

In [14]:
columns_to_encode = ['feature_score','furnish_score','floor','balcony','property_type']

In [15]:
column_trans=make_column_transformer(
    (OneHotEncoder(drop='first'),['age','property_type']),
    (StandardScaler(), ['bedRoom', 'bathroom', 'area']),
    (OrdinalEncoder(),columns_to_encode),
    (ce.TargetEncoder(), ['sector']),
    remainder='passthrough')

In [16]:
# model_output = []
for model_name,model in model_dict.items():
    # model_output.append(scorer(model_name, model))
    pipeline = make_pipeline(column_trans,model)
    pipeline.fit(X_train,y_train)
    y_pred = pipeline.predict(X_test)
    print(f" model is {model_name} and r2_score is {r2_score(y_test,y_pred)} and error is {mean_absolute_error(np.expm1(y_test),np.expm1(y_pred))}")

 model is linear_reg and r2_score is 0.6899177997135209 and error is 0.9110285902444801
 model is svr and r2_score is 0.7878469440896727 and error is 0.7095855050882758
 model is ridge and r2_score is 0.6901057364377665 and error is 0.9107413516681315
 model is LASSO and r2_score is -0.00021697241587892435 and error is 1.5827672722146149
 model is decision tree and r2_score is 0.7349023426095547 and error is 0.7580912328767123
 model is random forest and r2_score is 0.8663140853967944 and error is 0.5269405344140267
 model is extra trees and r2_score is 0.8708514724037564 and error is 0.5324036736296484
 model is gradient boosting and r2_score is 0.8474705467424719 and error is 0.5926403731307873
 model is adaboost and r2_score is 0.6437173774379934 and error is 0.9580216366206239
 model is mlp and r2_score is 0.7775685709237525 and error is 0.7345965635226813
 model is xgboost and r2_score is 0.8605461596662916 and error is 0.5542679350641002


### Ordinal Encoding

In [24]:
columns_to_encode = ['feature_score','furnish_score','floor','balcony','property_type','sector','age','property_type']

In [25]:
column_trans=make_column_transformer(
    # (OneHotEncoder(drop='first'),['age','property_type']),
    (StandardScaler(), ['bedRoom', 'bathroom', 'area']),
    (OrdinalEncoder(),columns_to_encode),
    # (ce.TargetEncoder(), ['sector']),
    remainder='passthrough')

In [26]:
for model_name,model in model_dict.items():
    pipeline = make_pipeline(column_trans,model)
    pipeline.fit(X_train,y_train)
    y_pred = pipeline.predict(X_test)
    print(f" model is {model_name} and r2_score is {r2_score(y_test,y_pred)} and error is {mean_absolute_error(np.expm1(y_test),np.expm1(y_pred))}")

 model is linear_reg and r2_score is 0.6193257184454812 and error is 1.0002171436074496
 model is svr and r2_score is 0.49713875347199254 and error is 1.0979007094049775
 model is ridge and r2_score is 0.6194359809792427 and error is 1.000290620922267
 model is LASSO and r2_score is 3.119590106459569e-06 and error is 1.5825669764685582
 model is decision tree and r2_score is 0.7021544331400753 and error is 0.7526230136986302
 model is random forest and r2_score is 0.8393371116688045 and error is 0.5768401551571867
 model is extra trees and r2_score is 0.8228650253331893 and error is 0.6129292465601976
 model is gradient boosting and r2_score is 0.833756867940254 and error is 0.6514611800708315
 model is adaboost and r2_score is 0.5474157857108699 and error is 1.1055483719033465
 model is mlp and r2_score is 0.7459711971531673 and error is 0.7568995553419873
 model is xgboost and r2_score is 0.8374143016297646 and error is 0.5751361351299938


### Final conclusions are 
- We try the 2nd encoding techniques and hyper parameters the models like
- RandomTree,ExtraTree,XGbooost

In [31]:
from sklearn.model_selection import GridSearchCV

rf = RandomForestRegressor(random_state=42)

param_grid = {
    'n_estimators': [200, 300, 500],
    'max_depth': [None, 20, 30, 40],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', 0.5],
    'bootstrap': [True]
}


grid = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=5,
    scoring='neg_mean_absolute_error',
    n_jobs=-1
)



In [32]:
column_trans=make_column_transformer(
    (OneHotEncoder(drop='first'),['age','property_type']),
    (StandardScaler(), ['bedRoom', 'bathroom', 'area']),
    (OrdinalEncoder(),columns_to_encode),
    (ce.TargetEncoder(), ['sector']),
    remainder='passthrough')

In [33]:
pipeline = make_pipeline(column_trans,grid)
pipeline.fit(X_train,y_train)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('columntransformer', ...), ('gridsearchcv', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('onehotencoder', ...), ('standardscaler', ...), ...]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'passthrough'
,"sparse_threshold sparse_threshold: float, default=0.3I

In [34]:
y_pred = pipeline.predict(X_test)
print(f" model is {model_name} and r2_score is {r2_score(y_test,y_pred)} and error is {mean_absolute_error(np.expm1(y_test),np.expm1(y_pred))}")

 model is xgboost and r2_score is 0.8842729085676122 and error is 0.4998901765040571


In [37]:
grid.best_params_

{'bootstrap': True,
 'max_depth': None,
 'max_features': 0.5,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 300}

In [ ]:
# {'bootstrap': True,
#  'max_depth': None,
#  'max_features': 0.5,
#  'min_samples_leaf': 1,
#  'min_samples_split': 2,
#  'n_estimators': 300}

In [39]:
etr = ExtraTreesRegressor(
    random_state=42,
    n_jobs=-1
)

param_grid = {
    'n_estimators': [300, 500, 800],
    'max_depth': [None, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': [0.3, 0.5, 'sqrt'],
    'bootstrap': [True, False]
}

grid = GridSearchCV(
    estimator=etr,
    param_grid=param_grid,
    cv=5,
    scoring='neg_mean_absolute_error',  # BEST for house prices
    n_jobs=-1
)

In [45]:
import time 
start = time.time()
print(f"Time taken to execute is is : {time.time()-start}")

Time taken to execute is is : 9.965896606445312e-05


In [46]:
start = time.time()
pipeline = make_pipeline(column_trans,grid)
pipeline.fit(X_train,y_train)
print(f"Time taken to execute is is : {time.time()-start}")

Time taken to execute is is : 1803.6193978786469


In [47]:
y_pred = pipeline.predict(X_test)
print(f" model is ExtraTree and r2_score is {r2_score(y_test,y_pred)} and error is {mean_absolute_error(np.expm1(y_test),np.expm1(y_pred))}")

 model is ExtraTree and r2_score is 0.8705035957076627 and error is 0.5378411077302064


In [48]:
grid.best_params_

{'bootstrap': False,
 'max_depth': 20,
 'max_features': 0.5,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 500}

In [ ]:
# {'bootstrap': False,
#  'max_depth': 20,
#  'max_features': 0.5,
#  'min_samples_leaf': 1,
#  'min_samples_split': 2,
#  'n_estimators': 500}

In [49]:
xgb = XGBRegressor(
    objective='reg:squarederror',
    random_state=42,
    n_jobs=-1
)

param_grid = {
    'n_estimators': [300, 500, 800],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'reg_alpha': [0, 0.1, 1],
    'reg_lambda': [1, 5, 10]
}


grid = GridSearchCV(
    estimator=xgb,
    param_grid=param_grid,
    cv=5,
    scoring='neg_mean_absolute_error',
    n_jobs=-1
)

In [50]:
start = time.time()
pipeline = make_pipeline(column_trans,grid)
pipeline.fit(X_train,y_train)
print(f"Time taken to execute is is : {time.time()-start}")

Time taken to execute is is : 1779.1511256694794


In [51]:
y_pred = pipeline.predict(X_test)
print(f" model is XGBoost and r2_score is {r2_score(y_test,y_pred)} and error is {mean_absolute_error(np.expm1(y_test),np.expm1(y_pred))}")

 model is XGBoost and r2_score is 0.8933376263731612 and error is 0.4735044208757844


In [52]:
grid.best_params_

{'colsample_bytree': 0.6,
 'learning_rate': 0.05,
 'max_depth': 7,
 'n_estimators': 800,
 'reg_alpha': 0.1,
 'reg_lambda': 1,
 'subsample': 0.8}

In [ ]:
# {'colsample_bytree': 0.6,
#  'learning_rate': 0.05,
#  'max_depth': 7,
#  'n_estimators': 800,
#  'reg_alpha': 0.1,
#  'reg_lambda': 1,
#  'subsample': 0.8}

In [53]:
import pickle

with open('pipeline.pkl', 'wb') as file:
    pickle.dump(pipeline, file)

In [54]:
with open('df.pkl', 'wb') as file:
    pickle.dump(X, file)

,area,property_type,bedRoom,bathroom,balcony,sector,Study_room,Servant_room,Pooja_room,floor,age,feature_score,furnish_score
0,5000.00,flat,2,2,1,Krishna Colony,0,0,0,Mid Floor,Relatively New,Low,Low
1,7692.00,flat,2,2,1,Ashok Vihar,0,0,0,Low Floor,Old,Low,Medium
2,6722.00,flat,2,2,3,Sohna,0,0,0,High Floor,Newly House,Medium,Unfurnished
3,12250.00,flat,2,2,2,61,1,0,0,Low Floor,Under Construction,Medium,Unfurnished
4,5204.00,flat,2,2,3,92,1,0,0,Mid Floor,Under Construction,Low,Unfurnished
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3643,1650.05,house,3,3,3,Sushant Lok Phase,0,0,0,Low Floor,Mid,Medium,Medium
3644,1799.97,house,4,4,3,26,1,1,1,Low Floor,Mid,Low,Low
3645,1350.02,house,3,2,3,Dlf Phase,1,0,0,Low Floor,Old,Low,Unfurnished
3646,1350.01,house,3,3,2,26,1,1,0,Low Floor,Mid,Medium,Medium


In [64]:
data = [[6722,'flat',2,2,'3','Sohna',0,0,0,'High Floor','Newly House','Medium','Unfurnished']]
columns = list(X.columns.values)

# Convert to DataFrame
one_df = pd.DataFrame(data, columns=columns)

one_df

,area,property_type,bedRoom,bathroom,balcony,sector,Study_room,Servant_room,Pooja_room,floor,age,feature_score,furnish_score
0,6722,flat,2,2,3,Sohna,0,0,0,High Floor,Newly House,Medium,Unfurnished


In [70]:
one_df=X.loc[3644]

In [91]:
one_df = df.loc[3644:]

In [95]:
one_df.iloc[2:3]

,area,property_type,bedRoom,bathroom,balcony,sector,Price,Study_room,Servant_room,Pooja_room,floor,age,feature_score,furnish_score
3646,1350.01,house,3,3,2,26,4.5,1,1,0,Low Floor,Mid,Medium,Medium


In [67]:
y

0       0.45
1       0.50
2       0.40
3       1.47
4       0.70
        ... 
3643    4.75
3644    5.50
3645    4.25
3646    4.50
3647    3.25
Name: Price, Length: 3648, dtype: float64